In [47]:
#################### Data Processing ######################
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

##################### Model building #####################
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [73]:
data="California is a state in the Western United States. California borders Oregon to the north, Nevada and Arizona to the east, the Mexican state of Baja California to the south; and has a coastline along the Pacific Ocean to the west."

data

'California is a state in the Western United States. California borders Oregon to the north, Nevada and Arizona to the east, the Mexican state of Baja California to the south; and has a coastline along the Pacific Ocean to the west.'

# Data Pre-Processing 

In [49]:
# cleaning the data
data= data.lower()           # Converting the string to lower case to get uniformity
data

'california is a state in the western united states. california borders oregon to the north, nevada and arizona to the east, the mexican state of baja california to the south; and has a coastline along the pacific ocean to the west.'

In [50]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [51]:
def remove_punctuation(text):
    text_nopunt="".join([c 
                         for c in text
                         if c not in string.punctuation])
    return text_nopunt

In [52]:
data=remove_punctuation(data)

data

'california is a state in the western united states california borders oregon to the north nevada and arizona to the east the mexican state of baja california to the south and has a coastline along the pacific ocean to the west'

In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
  
text_tokens = word_tokenize(data)
print("Word Token:  \n",text_tokens)

Word Token:  
 ['california', 'is', 'a', 'state', 'in', 'the', 'western', 'united', 'states', 'california', 'borders', 'oregon', 'to', 'the', 'north', 'nevada', 'and', 'arizona', 'to', 'the', 'east', 'the', 'mexican', 'state', 'of', 'baja', 'california', 'to', 'the', 'south', 'and', 'has', 'a', 'coastline', 'along', 'the', 'pacific', 'ocean', 'to', 'the', 'west']


In [55]:
tokens_without_sw = [word 
                     for word in text_tokens 
                     if not word in stopwords.words()]
print("Word Without StopWords:  \n",tokens_without_sw)

Word Without StopWords:  
 ['california', 'state', 'western', 'united', 'states', 'california', 'borders', 'oregon', 'north', 'nevada', 'arizona', 'east', 'mexican', 'state', 'baja', 'california', 'south', 'coastline', 'pacific', 'ocean', 'west']


In [56]:
data = (" ").join(tokens_without_sw)
data

'california state western united states california borders oregon north nevada arizona east mexican state baja california south coastline pacific ocean west'

In [57]:
# Instantiating the Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])  ## convert sentance to word  
sequence_data = tokenizer.texts_to_sequences([data])[0]  # mode concept for sequence 
sequence_data   

[1, 2, 3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 2, 13, 1, 14, 15, 16, 17, 18]

In [58]:
# Getting the total number of words of the data.
word2idx = tokenizer.word_index  ####### index number to every token ro word 
print(len(word2idx))

18


In [59]:
print(word2idx)

{'california': 1, 'state': 2, 'western': 3, 'united': 4, 'states': 5, 'borders': 6, 'oregon': 7, 'north': 8, 'nevada': 9, 'arizona': 10, 'east': 11, 'mexican': 12, 'baja': 13, 'south': 14, 'coastline': 15, 'pacific': 16, 'ocean': 17, 'west': 18}


In [60]:
vocab_size = len(word2idx) + 1    ### Get unique words
print(vocab_size)         # california = 0+1   

19


In [61]:
sequence_data

[1, 2, 3, 4, 5, 1, 6, 7, 8, 9, 10, 11, 12, 2, 13, 1, 14, 15, 16, 17, 18]

In [62]:
sequences = []  # empty list 
for i in range(3,len(sequence_data)): # i = 3 to 20 # sequence_data= count of words
    abc=sequence_data[i-3:i+1] # 0,1
    sequences.append(abc)

print(len(sequences))

18


In [63]:
sequences

[[1, 2, 3, 4],
 [2, 3, 4, 5],
 [3, 4, 5, 1],
 [4, 5, 1, 6],
 [5, 1, 6, 7],
 [1, 6, 7, 8],
 [6, 7, 8, 9],
 [7, 8, 9, 10],
 [8, 9, 10, 11],
 [9, 10, 11, 12],
 [10, 11, 12, 2],
 [11, 12, 2, 13],
 [12, 2, 13, 1],
 [2, 13, 1, 14],
 [13, 1, 14, 15],
 [1, 14, 15, 16],
 [14, 15, 16, 17],
 [15, 16, 17, 18]]

In [64]:
import numpy as np
sequences=np.array(sequences)
sequences

array([[ 1,  2,  3,  4],
       [ 2,  3,  4,  5],
       [ 3,  4,  5,  1],
       [ 4,  5,  1,  6],
       [ 5,  1,  6,  7],
       [ 1,  6,  7,  8],
       [ 6,  7,  8,  9],
       [ 7,  8,  9, 10],
       [ 8,  9, 10, 11],
       [ 9, 10, 11, 12],
       [10, 11, 12,  2],
       [11, 12,  2, 13],
       [12,  2, 13,  1],
       [ 2, 13,  1, 14],
       [13,  1, 14, 15],
       [ 1, 14, 15, 16],
       [14, 15, 16, 17],
       [15, 16, 17, 18]])

In [65]:
X = []
Y = []
for i in sequences: 
    X.append(i[0:3])  # i = 0,1,2
    Y.append(i[3])

X = np.array(X)
Y = np.array(Y)

In [66]:
print("Data" , X[:5])
print("Response" , Y[:5])

Data [[1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 1]
 [5 1 6]]
Response [4 5 1 6 7]


In [67]:
Y

array([ 4,  5,  1,  6,  7,  8,  9, 10, 11, 12,  2, 13,  1, 14, 15, 16, 17,
       18])

In [68]:
Y=to_categorical(Y,num_classes=vocab_size)  # Matrix of Y  
Y[:5]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype=float32)

# Model Building

In [69]:
model = Sequential()
model.add(Embedding(vocab_size,10, input_length=3)) # data import
model.add(LSTM(50,return_sequences=True)) # LSTM1
model.add(LSTM(50))  # LSTM2
model.add(Dense(50,activation='relu')) # HIDDEN LAYER
model.add(Dense(vocab_size, activation='softmax'))

In [70]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             190       
                                                                 
 lstm_2 (LSTM)               (None, 3, 50)             12200     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 19)                969       
                                                                 
Total params: 36,109
Trainable params: 36,109
Non-trainable params: 0
_________________________________________________________________


In [71]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [72]:
r = model.fit(X,Y,epochs=200)

Epoch 1/200
1/1 [==============================] - 18s 18s/step - loss: 2.9446 - accuracy: 0.0556
Epoch 2/200
1/1 [==============================] - 0s 39ms/step - loss: 2.9436 - accuracy: 0.0556
Epoch 3/200
1/1 [==============================] - 0s 42ms/step - loss: 2.9428 - accuracy: 0.0556
Epoch 4/200
1/1 [==============================] - 0s 25ms/step - loss: 2.9419 - accuracy: 0.0556
Epoch 5/200
1/1 [==============================] - 0s 46ms/step - loss: 2.9411 - accuracy: 0.0556
Epoch 6/200
1/1 [==============================] - 0s 34ms/step - loss: 2.9401 - accuracy: 0.1667
Epoch 7/200
1/1 [==============================] - 0s 37ms/step - loss: 2.9392 - accuracy: 0.1667
Epoch 8/200
1/1 [==============================] - 0s 66ms/step - loss: 2.9381 - accuracy: 0.1111
Epoch 9/200
1/1 [==============================] - 0s 31ms/step - loss: 2.9371 - accuracy: 0.1111
Epoch 10/200
1/1 [==============================] - 0s 26ms/step - loss: 2.9360 - accuracy: 0.1111
Epoch 11/200
1/1 [=

1/1 [==============================] - 0s 18ms/step - loss: 1.5898 - accuracy: 0.4444
Epoch 84/200
1/1 [==============================] - 0s 26ms/step - loss: 1.5570 - accuracy: 0.5556
Epoch 85/200
1/1 [==============================] - 0s 22ms/step - loss: 1.5238 - accuracy: 0.6111
Epoch 86/200
1/1 [==============================] - 0s 20ms/step - loss: 1.4907 - accuracy: 0.6111
Epoch 87/200
1/1 [==============================] - 0s 22ms/step - loss: 1.4582 - accuracy: 0.6111
Epoch 88/200
1/1 [==============================] - 0s 38ms/step - loss: 1.4260 - accuracy: 0.6111
Epoch 89/200
1/1 [==============================] - 0s 39ms/step - loss: 1.3940 - accuracy: 0.6111
Epoch 90/200
1/1 [==============================] - 0s 27ms/step - loss: 1.3622 - accuracy: 0.6667
Epoch 91/200
1/1 [==============================] - 0s 37ms/step - loss: 1.3311 - accuracy: 0.6667
Epoch 92/200
1/1 [==============================] - 0s 25ms/step - loss: 1.3006 - accuracy: 0.7222
Epoch 93/200
1/1 [=====

1/1 [==============================] - 0s 21ms/step - loss: 0.0753 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 26ms/step - loss: 0.0726 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 27ms/step - loss: 0.0701 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 33ms/step - loss: 0.0677 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 29ms/step - loss: 0.0654 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 32ms/step - loss: 0.0633 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 39ms/step - loss: 0.0612 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 43ms/step - loss: 0.0592 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 26ms/step - loss: 0.0573 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 28ms/step - loss: 0.0555 - accuracy: 1.0000
Epoch 175/200


# Future Word Prediction

#### Describe
* Model = model we build
* tokenizer = Breaking into Word
* enter_text = input user give

In [39]:
for word, index in tokenizer.word_index.items():
    print(word,index)

california 1
state 2
western 3
united 4
states 5
borders 6
oregon 7
north 8
nevada 9
arizona 10
east 11
mexican 12
baja 13
south 14
coastline 15
pacific 16
ocean 17
west 18


In [40]:
enter_text='states california borders'
encoded = tokenizer.texts_to_sequences([enter_text]) 
encoded = np.array(encoded) # [3]
predicted= np.argmax(model.predict(encoded))  # input = 3 # prediction = 4  
predicted

1/1 [==============================] - 3s 3s/step


7

In [41]:
def Predict_Next_Words(model,tokenizer,enter_text):
        encoded = tokenizer.texts_to_sequences([enter_text]) # in_text = Western = 3 
        encoded = np.array(encoded) # [3]
        predicted= np.argmax(model.predict(encoded))  # input = 3 # prediction = 4  
        predicted_word=''
        for word, index in tokenizer.word_index.items():
            if  index==predicted:    # Predicted  7 = Index 7
                predicted_word = word      # word = OREGON
                break
        result=enter_text + ' ' + predicted_word
        return result

In [42]:
data

'california state western united states california borders oregon north nevada arizona east mexican state baja california south coastline pacific ocean west'

In [43]:
print(Predict_Next_Words(model,tokenizer,'states california borders'))

1/1 [==============================] - 0s 38ms/step
states california borders oregon


In [44]:
print(Predict_Next_Words(model,tokenizer,'california borders oregon'))

1/1 [==============================] - 0s 56ms/step
california borders oregon north


In [45]:
print(Predict_Next_Words(model,tokenizer,'borders oregon north'))

1/1 [==============================] - 0s 57ms/step
borders oregon north nevada


In [46]:
print(Predict_Next_Words(model,tokenizer,'oregon north nevada '))

1/1 [==============================] - 0s 55ms/step
oregon north nevada  arizona


# Finished